In [18]:
import pandas as pd
from google.colab import files

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', '{:.2f}'.format)

# Carregar o arquivo CSV
uploaded = files.upload()

# Obter o nome do arquivo carregado
uploaded_filename = list(uploaded.keys())[0]

# Carregar o DataFrame
df = pd.read_csv(uploaded_filename)

states = {
    "AC": "Acre", "AL": "Alagoas", "AP": "Amapá", "AM": "Amazonas", "BA": "Bahia", "CE": "Ceará",
    "DF": "Distrito Federal", "ES": "Espírito Santo", "GO": "Goiás", "MA": "Maranhão", "MT": "Mato Grosso",
    "MS": "Mato Grosso do Sul", "MG": "Minas Gerais", "PA": "Pará", "PB": "Paraíba", "PR": "Paraná",
    "PE": "Pernambuco", "PI": "Piauí", "RJ": "Rio de Janeiro", "RN": "Rio Grande do Norte",
    "RS": "Rio Grande do Sul", "RO": "Rondônia", "RR": "Roraima", "SC": "Santa Catarina", "SP": "São Paulo",
    "SE": "Sergipe", "TO": "Tocantins"
}

products = [
    {'id': 1, 'name': 'CANA-DE-AÇUCAR', 'states': ('SP', 'MG', 'ES', 'RJ', 'PE', 'AL', 'PB', 'SE', 'RN', 'BA', 'CE',
                                                   'PI', 'PR', 'RS', 'SC', 'MS', 'MT', 'GO', 'RS', 'PR', 'SC')},
    {'id': 2, 'name': 'BATATA-DOCE', 'states': (
        'SP', 'MG', 'RJ', 'RS', 'PR', 'SC', 'SE', 'CE', 'AL', 'RN', 'PE', 'GO', 'DF')},
    {'id': 3, 'name': 'GUARANÁ', 'states': ('BA', 'AM', 'AC')},
    {'id': 4, 'name': 'MANDIOCA (AIPIM, MACAXEIRA)', 'states': ('BA', 'AM', 'AC')}
]


def get_product_name_by_id(product_types, product_id):
    return next(p['name'] for p in product_types if p['id'] == product_id)


def get_products_types(product_types):
    message = ''
    for product_type in product_types:
        message += f"\n{product_type['id']} - {product_type['name']}"
    return message


def get_uf_by_products_types(product_types, product_id):
    ufs = product_types[product_id - 1]['states']
    message = ''
    for uf in ufs:
        message += f"\n{uf} - {states[uf]}"
    return message


def check_product(product_id, product_types):
    try:
        number = int(product_id)
        if number not in [item['id'] for item in product_types]:
            raise ValueError()
        return True, number
    except ValueError:
        return False, None


def check_uf(uf, product_id, product_types):
    try:
        ufs = product_types[product_id - 1]['states']
        if uf not in ufs:
            raise ValueError()
        return True, uf
    except ValueError:
        return False, None


print("\nOlá Seja bem vindo ao sistema AgroFácil. \nVamos te ajudar a descobrir "
      "se seu produto tem boas chances de conseguir crédito rural, de acordo com sua região e histórico.\n")

# Selecionar produto
while True:
    is_valid, product = check_product(
        input("Selecione o tipo de sua produção: " + get_products_types(products)).upper(),
        products
        )
    if not is_valid:
        print('\nOPÇÃO INVÁLIDA. TENTE NOVAMENTE!!!')
    else:
        break

# Selecionar UF
while True:
    is_valid, uf = check_uf(
        input("\nAgora, informe a sigla do seu estado: " + get_uf_by_products_types(products, product)).upper(),
        product, products)
    if not is_valid:
        print('\n\nOPÇÃO INVÁLIDA. TENTE NOVAMENTE!!!')
    else:
        break

product_name = get_product_name_by_id(products, product)

# product_name = 'CANA-DE-AÇUCAR'
# uf = "SP"

df.rename(columns={'Produto1': 'Produto'}, inplace=True)

# Descobrir o percentual de confiabilidade de crédito do produto escolhido pelo usuário
# cria um novo dataframe a partir de uma cópia de df
filterByProductAndUF = df[(df['Produto'] == product_name) & (df['UF'] == uf)].copy()

filterByProductAndUF['adesoes'] = filterByProductAndUF['Textbox78'].replace('[\.]', '', regex=True).str.replace(',',
                                                                                                                '.').astype(
    float)
filterByProductAndUF['perdas'] = filterByProductAndUF['Textbox82'].replace('[\.]', '', regex=True).str.replace(',',
                                                                                                               '.').astype(
    float)
sinister = filterByProductAndUF['perdas'] / filterByProductAndUF['adesoes']
filterByProductAndUF['confiabilidade'] = (1 - sinister) * 100

# percentual de confiabilidade do produto escolhido pelo usuário
productScore = round(filterByProductAndUF[['confiabilidade']].iloc[0, 0], 2)

print('\n\n')

if productScore >= 80:
    print(f'O seu produto tem um alto score de {productScore}%.')
    response = 'Caso queira diversificar sua  oferta de produtos, sugerimos os seguintes em sua região:'
elif productScore >= 70:
    print(f'O seu produto tem um score médio de {productScore}%.')
    response = 'Caso queira diversificar sua oferta de produtos, sugerimos os seguintes em sua região:'
elif productScore >= 40:
    print(f'O seu produto tem um baixo score de {productScore}%.')
    response = 'Recomendamos que avalie a  mudança de produto para outros com maior possibilidade de crédito em sua ' \
               'região:'
else:
    print(f'O seu produto tem um baixíssimo score de {productScore}%.')
    response = 'Recomendamos ' \
               'fortemente a mudança de ' \
               'produto para outros com maior possibilidade de crédito em sua região:'

# Selecionar os produtos por estado e confiabilidade
filterByUf = df[df['UF'] == uf].copy()
filterByUf['adesoes'] = filterByUf['Textbox78'].replace('[\.]', '', regex=True).str.replace(',', '.').astype(float)
filterByUf['perdas'] = filterByUf['Textbox82'].replace('[\.]', '', regex=True).str.replace(',', '.').astype(float)
sinister = filterByUf['perdas'] / filterByUf['adesoes']
filterByUf['score %'] = (1 - sinister) * 100

# remover valores com baixa expressividade e o próprio produto do usuário
filterByUf = filterByUf[(filterByUf['perdas'] > 0) & (df['Produto'] is not product_name)]

# filtrar apenas produtos com bom score
filterByUf = filterByUf[filterByUf['score %'] > 70]

# Cria dataframe produtos por estado e score%
listProductPct = filterByUf[['Produto', 'score %']]

# ordenar por score%
listProductPct = listProductPct.sort_values(by='score %', ascending=False)

# Remove a primeira coluna com os indices
listProductPct.reset_index(drop=True, inplace=True)

listProductPct.index += 1

total_linhas = listProductPct.shape[0]

if total_linhas > 0:
    print(response)
    print(listProductPct)
else:
    print("Infelizmente nossa base de dados não tem informações suficiente para lhe oferecer outras opçoes de produtos "
          "em sua região.")

Saving proagro.csv to proagro (11).csv

Olá Seja bem vindo ao sistema AgroFácil. 
Vamos te ajudar a descobrir se seu produto tem boas chances de conseguir crédito rural, de acordo com sua região e histórico.


Agora, informe a sigla do seu estado: 
BA - Bahia
AM - Amazonas
AC - Acream



O seu produto tem um alto score de 100.0%.
Infelizmente nossa base de dados não tem informações suficiente para lhe oferecer outras opçoes de produtos em sua região.
